In [1]:
import pyodbc
import nltk
import pandas as pd
import numpy as np
import re
from pandas.io import sql
from collections import defaultdict
from nltk.corpus import wordnet as wn
import csv
from scipy import spatial
from PyDictionary import PyDictionary


# fuction for tag sentence

In [2]:
def tag(text):
    text = nltk.word_tokenize(text)
    return [list(i) for i in zip(*nltk.pos_tag(text))] 
    # output a list like [['it', 'is', 'easy', 'to'], ['PRP', 'VBZ', 'JJ', 'TO']]

## functions for finding pattern in sentence


mfind is to find patterns like "it is easy to install"

mfind2 is to find pattens like "it is great"

In [3]:
def mfind(tr,ts,rep):
    ## this is to find the meaningful sentence in the comments using given pattern rep
    # tr, ts is two list from tag, rep is a pattern from
    m_list = []
    p_list = []
    #this is the list of meaningful sentence
    ts_string = "+".join(ts) # ts joined by "+", so could be used to find pattern
    pattern_lst = rep.findall(ts_string)
    for pattern in pattern_lst:
        #pattern is a string like 'PRP+VBZ+JJ'
        tmp = pattern.split('+')
        plen = len(tmp)
        start = tmp[0] # get the first element of the pattern
        for idx,item in enumerate(ts):
            #print item,tmp,ts[idx:idx+plen-1] 
            if (item == start) and (ts[idx:idx+plen] == tmp):
                m_list.append(tr[idx:idx+plen])
                p_list.append(tmp)
    return m_list,p_list
    # m_list is a list of list [['It', "'s", 'stylish'], ['it', 'was', 'great']]
    # p_list is also the same but like [['JJ', 'TO', 'VB'],['JJ', 'TO', 'VB']]


In [4]:
def mfind2(tr,ts,rep):
    ## this is to find the meaningful sentence in the comments using given pattern rep
    # tr, ts is two list from tag, rep is a pattern from
    m_list = []
    p_list = []
    #this is the list of meaningful sentence
    ts_string = "+".join(ts) # ts joined by "+", so could be used to find pattern
    pattern_lst = rep.findall(ts_string)
    for pattern_tuple in pattern_lst:
        #pattern_lst is like [('', 'NN+VBD+JJ', ''), ('', 'NNP+VBN+JJ', ''), ('', '', 'NNP+VBZ+JJ')]
        for pattern in pattern_tuple:
            if pattern !='':
                #pattern is a string like 'PRP+VBZ+JJ'
                tmp = pattern.split('+')
                plen = len(tmp)
                start = tmp[0] # get the first element of the pattern
                for idx,item in enumerate(ts):
                    #print item,tmp,ts[idx:idx+plen-1] 
                    if (item == start) and (ts[idx:idx+plen] == tmp):
                        m_list.append(tr[idx:idx+plen])
                        p_list.append(tmp)
    return m_list,p_list
    # m_list is a list of list [['It', "'s", 'stylish'], ['it', 'was', 'great']]
    # p_list is also the same but like [['JJ', 'TO', 'VB'],['JJ', 'TO', 'VB']]


# get the data
see the folder

# data cleaning

In [7]:
comments = df.PrvPrComments.values
comments = [re.split('[,.!?;]',i) for i in comments if i >0]

comments = reduce(lambda x,y:x+y,comments)
comments = [i.strip() for i in comments if i != ''];comments = [i for i in comments if i != '']
# convert them into lower case
comments = [i.lower() for i in comments]

In [8]:
## save data 
pd.to_pickle(comments,'d:\comments for tag')

In [ ]:
## load data from pickle
comments = pd.read_pickle(r'd:\comments for tag')

# syntax analysis and pattern find

In [9]:
# pattern for something is somelike
rep =re.compile(r'(PRP\+VB[DPZN]\+JJ)\+[^T]|(NNP?S?\+VB[DPZN]\+JJ)\+[^T]|(NNP?S?\+VB[DPZN]\+JJ)$')
#pattern for somelike to do something
rep2 = re.compile('NNP?S?\+VB[DPZN]\+JJ\+TO\+VB\+RB|NNP?S?\+VB[DPZN]\+JJ\+TO\+VB|JJ\+TO\+VB\+RB|JJ\+TO\+VB')

In [10]:
pm1 = []
ps1 = []
pm2 = []
ps2 = []
for i in comments:
    #print i
    tr,ts = tag(i)
    m,s = mfind2(tr,ts,rep)
    if bool(m):
        pm1+=m
        ps1+=s
    m,s = mfind(tr,ts,rep2)
    if bool(m):
        pm2+=m
        ps2+=s

In [12]:
pm1[:5],ps1[:5],pm2[:5],ps2[:5]

([[u'gray', u'is', u'nice'],
  [u'it', u"'s", u'such'],
  [u'bed', u'is', u'excellent'],
  [u'i', u'was', u'careful'],
  [u'headboard', u'is', u'adjustable']],
 [['NN', 'VBZ', 'JJ'],
  ['PRP', 'VBZ', 'JJ'],
  ['NN', 'VBZ', 'JJ'],
  ['NN', 'VBD', 'JJ'],
  ['NN', 'VBZ', 'JJ']],
 [[u'able', u'to', u'put'],
  [u'easy', u'to', u'put', u'together'],
  [u'easy', u'to', u'put', u'together'],
  [u'much', u'to', u'bedroom'],
  [u'negative', u'to', u'say']],
 [['JJ', 'TO', 'VB'],
  ['JJ', 'TO', 'VB', 'RB'],
  ['JJ', 'TO', 'VB', 'RB'],
  ['JJ', 'TO', 'VB'],
  ['JJ', 'TO', 'VB']])

In [13]:
from collections import Counter,defaultdict

In [14]:
print Counter(map(len,pm1)).most_common()
print Counter(map(len,pm2)).most_common()

[(3, 327)]
[(3, 128), (4, 85), (5, 20), (6, 6)]


# build the obj and adj list from given pattern

In [15]:
d1 = defaultdict(list)
for o,i,a in pm1:
    d1[o.lower()].append(a)

In [16]:
d1

defaultdict(list,
            {u'assembly': [u'easy',
              u'easy',
              u'easy',
              u'easy',
              u'easy',
              u'easy'],
             u'beautiful': [u'frame'],
             u'bed': [u'excellent',
              u'great',
              u'beautiful',
              u'funny',
              u'amazing',
              u'great',
              u'gorgeous',
              u'beautiful',
              u'pretty',
              u'great',
              u'beautiful',
              u'gorgeous',
              u'fantastic',
              u'great',
              u'sturdy',
              u'sturdy',
              u'sturdy',
              u'great',
              u'beautiful',
              u'great',
              u'perfect',
              u'sturdy',
              u'big',
              u'beautiful',
              u'nice',
              u'perfect',
              u'beautiful',
              u'sturdy',
              u'beautiful',
              u'wonderful',
        

# function generate the graph

In [17]:
d = PyDictionary()

In [18]:
def graph_gen_w(word_list,extend_synonym_len = 2):
    word_syn_pair = []
    for word in word_list:
        tmp = []
        try:
            s = d.synonym(word)
        except:
            pass
        else:
            #print word
            if s>0:
                for sword in s[:extend_synonym_len]:
                    try:
                        tmp += d.synonym(sword)
                    except:
                        pass
                s = s+tmp+[word]
                word_syn_pair.append((word,set(s)))
        #return word_syn_pair
    ## generate the graph from here
    dic = defaultdict(dict)
    for idx,item in enumerate(word_syn_pair):
        dic[item[0]][item[0]] = 0
        for idx2,item2 in enumerate(word_syn_pair[idx+1:]):
            inter = item[1].intersection(item2[1])
            if bool(inter):
                dic[item[0]][item2[0]]= len(inter)
                dic[item2[0]][item[0]]= len(inter)
    return dic,word_syn_pair

In [19]:
d2 = [(key,len(d1[key]) )for key in d1]   
## add len() filter here to get top K 可以在这里增加筛选，第二步排序会更快，且可以无论长短都取前K


In [20]:
d2 = sorted(d2,key = lambda x: x[1],reverse = 1);d2[:10]

[(u'bed', 73),
 (u'it', 47),
 (u'i', 28),
 (u'fabric', 16),
 (u'color', 14),
 (u'headboard', 14),
 (u'price', 13),
 (u'quality', 10),
 (u'shipping', 8),
 (u'delivery', 7)]

In [22]:
obj = 'bed';d1[obj]

[u'excellent',
 u'great',
 u'beautiful',
 u'funny',
 u'amazing',
 u'great',
 u'gorgeous',
 u'beautiful',
 u'pretty',
 u'great',
 u'beautiful',
 u'gorgeous',
 u'fantastic',
 u'great',
 u'sturdy',
 u'sturdy',
 u'sturdy',
 u'great',
 u'beautiful',
 u'great',
 u'perfect',
 u'sturdy',
 u'big',
 u'beautiful',
 u'nice',
 u'perfect',
 u'beautiful',
 u'sturdy',
 u'beautiful',
 u'wonderful',
 u'great',
 u'awesome',
 u'beautiful',
 u'gorgeous',
 u'awesome',
 u'incredible',
 u'beautiful',
 u'gorgeous',
 u'gorgeous',
 u'awesome',
 u'great',
 u'sturdy',
 u'gorgeous',
 u'beautiful',
 u'beautiful',
 u'big',
 u'big',
 u'nice',
 u'awesome',
 u'great',
 u'solid',
 u'beautiful',
 u'perfect',
 u'sturdy',
 u'gorgeous',
 u'gorgeous',
 u'great',
 u'ok',
 u'sooooooo',
 u'gorgeous',
 u'sturdy',
 u'great',
 u'great',
 u'pretty',
 u'great',
 u'nice',
 u'fantastic',
 u'awesome',
 u'beautiful',
 u'sexy',
 u'awesme',
 u'beautiful',
 u'great']

In [23]:
graph,word_synlist = graph_gen_w(set(d1[obj]))

C:\Users\sjiang\AppData\Local\Continuum\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Error: A Term must be only a single word


In [24]:
# save the graph and word synlist
pd.to_pickle(graph,'graph for bed')
pd.to_pickle(word_synlist,'word_synlist for bed')

In [25]:
graph

defaultdict(dict,
            {u'amazing': {u'amazing': 0,
              u'awesome': 4,
              u'beautiful': 2,
              u'fantastic': 2,
              u'incredible': 6,
              u'pretty': 2,
              u'sexy': 2,
              u'wonderful': 3},
             u'awesme': {u'awesme': 0},
             u'awesome': {u'amazing': 4,
              u'awesome': 0,
              u'beautiful': 6,
              u'excellent': 2,
              u'gorgeous': 2,
              u'perfect': 2,
              u'pretty': 6,
              u'sexy': 2,
              u'sturdy': 1,
              u'wonderful': 2},
             u'beautiful': {u'amazing': 2,
              u'awesome': 6,
              u'beautiful': 0,
              u'excellent': 2,
              u'gorgeous': 1,
              u'perfect': 1,
              u'pretty': 10,
              u'sexy': 3,
              u'wonderful': 1},
             u'big': {u'big': 0, u'excellent': 1, u'gorgeous': 1, u'great': 8},
             u'excellent': 

# find out all the connect set for given graph

In [26]:
def dfs2(graph, start, weight=0):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            neibor = graph[vertex]
            candy = set([i[0] for i in neibor.items() if i[1]>weight])
            stack.extend(candy - visited)
    return visited

In [30]:
f = set(graph.keys());f

{u'amazing',
 u'awesme',
 u'awesome',
 u'beautiful',
 u'big',
 u'excellent',
 u'fantastic',
 u'funny',
 u'gorgeous',
 u'great',
 u'incredible',
 u'nice',
 u'ok',
 u'perfect',
 u'pretty',
 u'sexy',
 u'solid',
 u'sooooooo',
 u'sturdy',
 u'wonderful'}

# fuction to find connect set in the graph

In [31]:
### package it into a function

f = set(graph.keys())
group_list = []
while bool(f):
    t = f.pop()
    c = dfs2(graph,t,1)
    
    f = f-c
    group_list.append(c)
    print c

set([u'beautiful', u'perfect', u'fantastic', u'incredible', u'awesome', u'gorgeous', u'amazing', u'wonderful', u'pretty', u'excellent', u'sexy'])
set([u'funny'])
set([u'big', u'great'])
set([u'ok', u'nice'])
set([u'solid', u'sturdy'])
set([u'awesme'])
set([u'sooooooo'])


# find representive word using frequent

In [32]:
def rep_word(obj,group_list,filt = 0.1):
    c = Counter(d1[obj])
    total = sum(c.values())
    print total
    for group in group_list:
        group = list(group)
        max_word = group[0]
        max_count = c[group[0]]
        grp_count = 0
        
        for word in group:
            word_count = c[word]
            grp_count +=word_count
            if word_count > max_count:
                max_word = word
                max_count = word_count
        print group,grp_count
        if grp_count > total * filt:
            print max_word

In [33]:
rep_word(obj,group_list,filt = 0.05)

73
[u'beautiful', u'perfect', u'fantastic', u'incredible', u'awesome', u'gorgeous', u'amazing', u'wonderful', u'pretty', u'excellent', u'sexy'] 40
beautiful
[u'funny'] 1
[u'big', u'great'] 17
great
[u'ok', u'nice'] 4
nice
[u'solid', u'sturdy'] 9
sturdy
[u'awesme'] 1
[u'sooooooo'] 1
